# BLEU
**BLEU (Bilingual Evaluation Understudy)**

* 기계 번역 품질 평가 지표 중 하나이다.
* 생성된 문장과 여러 개의 기준(reference) 문장 간의 n-그램 일치 비율(정밀도)을 계산한다.
* 짧은 답만 생성해 일치율이 높아지는 것을 방지하기 위해 길이 벌점(Brevity Penalty)을 적용한다.
* 0\~1 사이 값을 가지며, 값이 클수록 기준 문장과 유사도가 높다.


* **Reference**:
  나는 학교에 갔다
* **Candidate**:
  나는 도서관에 갔다

1. 토큰화 (공백 단위):

   * Reference tokens = \[나는, 학교에, 갔다]
   * Candidate tokens = \[나는, 도서관에, 갔다]
2. **정밀도(precision)**
   겹치는 토큰 = {나는, 갔다} → 2개
   Precision = 2 / 3 ≈ 0.67
3. **길이 벌점(Brevity Penalty, BP)**
   Candidate 길이 = 3, Reference 길이 = 3 → BP = 1
4. **BLEU-1** = BP × Precision = 1 × 0.67 ≈ 0.67

> 이처럼 단어 1-그램 일치율을 재는 것이 BLEU-1이다.


**BLEU-N 구분**

* BLEU-1/2: 각각 단순 단어 정확도(1그램)나 2그램까지의 일치만 볼 때 사용하며, 짧은 문장 혹은 어순보다는 용어 사용만 보고 싶을 때 유용하다.
* BLEU-4: 1~4그램을 모두 고려하므로 번역 품질을 가장 종합적으로 평가할 때 주로 사용한다.




**점수구분**

BLEU-4 점수는 0에서 1 사이로 나오고, 보통 퍼센트(0–100)로 환산해 쓰기도 한다.언어 쌍(pairs)·데이터 특성·평가 셋에 따라 달라질수 있지만 대략적인 기준은 다음과 같다.

* **0.10 이하**: 아직 모방력이 많이 부족한 단계
* **0.10∼0.20**: 기초적인 단어 선택·순서는 맞추지만, 문법·표현 다양성은 떨어지는 수준
* **0.20∼0.30**: 평균적인 신경망 번역모델 성능대 (낮은 자원 언어쌍이나 도메인 특수 텍스트)
* **0.30∼0.40**: 보통의 오픈소스·상용 엔진이 도달하는 수준
* **0.40 이상**: 상당히 좋은 품질. 문장 구조·어휘 선택에서 레퍼런스와 높은 유사도를 보임
* **0.50 이상**: 최첨단(SOTA) 성능 혹은 평가 셋이 쉬운 경우에나 가능한 수준

실제로 논문·경쟁대회(WMT 등)에서는 영어↔유럽권 언어 쌍 기준으로 보통 **30∼40 BLEU**(0.30∼0.40)를 “괜찮다” 수준으로 보고, 40 이상이면 “매우 좋다”고 평가한다.
응용 중인 언어쌍과 데이터셋에 맞춰 이 범위를 참고하면 된다.


## sentence_bleu

In [1]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# reference 정답데이터(n개)
references = [
    "나는 학교에 갔다".split(),
    "나는 스쿨에 갔다".split(),
    "나는 수업에 갔다".split()
]

# candidate 기계번역 결과 : 모델이 생성한 번역 문장 (후보)
candidate = '나는 도서관에 갔다'.split()

smoothing_fn = SmoothingFunction().method1  # 문장이 짧거나 n-gram이 없을때 0점 방지용 스무딩

# unigram기반 bleu-1
bleu1 = sentence_bleu(
    references,
    candidate,
    weights=(1, 0, 0, 0), # unigram, bigram, trigram, 4-gram (1-gram만 적용)
    smoothing_function=smoothing_fn
)
print(bleu1)

# bigram기반 bleu-2
bleu2 = sentence_bleu(
    references,
    candidate,
    weights=(0.5, 0.5, 0, 0), # unigram, bigram, trigram, 4-gram (1-gram + 2-gram)
    smoothing_function=smoothing_fn
)
print(bleu2)

# bigram기반 bleu-4
bleu4 = sentence_bleu(
    references,
    candidate,
    weights=(0.25, 0.25, 0.25, 0.25), # unigram, bigram, trigram, 4-gram (default)
    smoothing_function=smoothing_fn
)
print(bleu4)

0.6666666666666666
0.18257418583505536
0.13512001548070346


n-gram이 길어질수록 (특히 3~4gram) 정답과 표현이 정확히 이어져야 점수가 올라간다.  
더 엄격하게 평가된다.  
기본 bleu 스코어는 4-gram 기준으로 평가하는 경우가 많다.

## corpus_bleu

In [2]:
from nltk.translate.bleu_score import corpus_bleu  # 문장 여러 개에 대한 BLEU(코퍼스 BLEU)

# 후보 문장 리스트 : 모델이 생성한 번역 후보들
candidates = [
    "잔돈은 가지세요.",
    "버스를 타자.",
    "당황하지 말자.",
    "삶이 쉽지만은 않잖아.",
    "들어가도 되겠습니까?",
    "내 가방이 비어 있는데.",
    "제대로 피가 끓어올랐다.",
    "내 고양이는 블랙이다.",
    "집에 아무도 없었다."
]

# 다중 레퍼런스 리스트 : 각 후보 문장에 대응하는 정답(레퍼런스) 문장들 (복수)
references = [
    ["잔돈은 가져가세요.", "거스름돈은 됐어요.", "잔돈은 필요 없어요."],
    ["버스를 타고 가자.", "버스로 가자.", "버스로 이동하자."],
    ["당황하지 말자.", "당황하지 마.", "허둥대지 말자."],
    ["삶이 쉽지만은 않다.", "인생은 만만치 않다.", "인생이 녹록지 않다."],
    ["들어가도 될까요?", "들어와도 괜찮을까요?", "제가 들어가도 될까요?"],
    ["내 가방이 비어 있다.", "내 가방엔 아무것도 없다.", "내 가방이 텅 비어 있다."],
    ["피가 끓어올랐다.", "분노가 치밀었다.", "혈기가 끓었다."],
    ["내 고양이는 검은색이다.", "내 고양이는 검은색 고양이야.", "내 고양이 털은 까맣다."],
    ["집에 아무도 없었다.", "집안에는 사람이 없었다.", "집에는 아무도 없었다."]
]

# 토큰화
tokenized_cand = [cand.split() for cand in candidates]  # 후보 문장들을 공백 기준 토큰 리스트로 변환
tokenized_ref = [[ref.split() for ref in refs] for refs in references ]  # (문장별) 다중 레퍼런스도 토큰 리스트로 변환

print(len(tokenized_cand), len(tokenized_ref))

# bleu-4 : 코퍼스 단위 BLEU-4 계산
bleu4 = corpus_bleu(
    tokenized_ref,    # 레퍼런스: [ [ref1_tokens, ref2_tokens, ...], ... ]
    tokenized_cand,   # 후보 : [cand_tokens, cand_tokens, ...]
    weights=(0.25, 0.25, 0.25, 0.25),
    smoothing_function=smoothing_fn
)
print(bleu4)

9 9
0.17021819207127004


값이 높을수록 레퍼런스와 모델의 번역 결과의 n-gram 일치가 많다고 해석한다.  
1에 가까울수록 완벽. 최상은 0.5로 판단